You will need to install matplotlib, seaborn, pandas and mpld3 to run this

In [ ]:
# for macOS
# !pip3 install --user matplotlib seaborn pandas mpld3
pip install matplotlib seaborn pandas mpld3

# Plotting Results

In [ ]:
import argparse
import csv
import matplotlib.pyplot as plt
import glob
import os
import json
import seaborn as sns
import pandas as pd
from plot_fitness import *
%matplotlib inline
import mpld3
from IPython import display
mpld3.enable_notebook()

In [ ]:
leonhard_directory = "compare_naive_to_island_Nov_13_143621"

In [ ]:
experiments, dataframes = extract_all_run_values(leonhard_directory)

In [ ]:
ax = sns.lineplot(x="epoch", y="fitness", hue="run", data=dataframes[0])
ax.set_title(experiments[0])
ax

In [ ]:
ax = sns.lineplot(x="epoch", y="fitness", hue="run", data=dataframes[1])
ax.set_title(experiments[1])
ax